In [ ]:
import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url

# Configuration       
cloudinary.config( 
    cloud_name = "dzr5vt5ut", 
    api_key = "829673595861745", 
    api_secret = "9rWMNm7vKvVMBEPzqiD2E8-YALE", # Click 'View Credentials' below to copy your API secret
    secure=True
)

# Upload an image
upload_result = cloudinary.uploader.upload("D:\\Work and Assignments\\Python\\Python Projects\\Google Vision API\\dp.jpg")
print(upload_result["secure_url"])


In [ ]:
c

In [ ]:
c.iloc[0]['product_url']

In [ ]:
fea = c['Extracted_Details_from_Image']
fea

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [ ]:
new_df=c
new_df.shape

In [ ]:
a="Brand: Reebok, Description: ['Polo shirt', 'Arm', 'Shoulder', 'Sports uniform', 'Jersey', 'Azure', 'Sleeve', 'Gesture', 'Sports jersey', 'Collar'], Text: Reebok, Object: ['Person', 'Top', 'Person', 'Top', 'Pants']"
ans=[]
for i in range(len(fea)):
    b=str(fea[i])

    a_vals = Counter(a)
    b_vals = Counter(b)

    # convert to word-vectors
    words  = list(a_vals.keys() | b_vals.keys())
    a_vect = [a_vals.get(word, 0) for word in words]       
    b_vect = [b_vals.get(word, 0) for word in words]        
    sim = cosine_similarity([a_vect], [b_vect])
    # print(f"{i+1}: {cosine_similarity([a_vect], [b_vect])}")
    new_df.loc[i, "SIM"] = sim[0][0]


In [ ]:
new_df

: 

In [ ]:
sort_df = new_df.sort_values(['SIM'],ascending=False,axis=0)

In [ ]:
sort_df

In [ ]:
import pandas as pd
c = pd.read_csv("D:\\Work and Assignments\\Python\\Product_DB.csv")
c.shape